In [ ]:
#https://medium.com/@tskumar1320/how-to-fine-tune-pre-trained-language-translation-model-3e8a6aace9f extensively used
#https://saturncloud.io/blog/how-to-use-huggingfaces-t5-model-for-translation-tasks-a-comprehensive-guide/ t5 guide
#https://towardsdatascience.com/how-to-adapt-a-multilingual-t5-model-for-a-single-language-b9f94f3d9c90 t5 guide better
! protoc --python_out=. sentencepiece_model.proto

In [2]:
! wget https://raw.githubusercontent.com/google/sentencepiece/master/src/sentencepiece_model.proto

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [32]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[30273, 261, 714, 1371, 259, 98923, 309, 1], [1494, 339, 259, 7845, 259, 98923, 260, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]}


In [4]:
print(transformers.__version__)
#below options for wandb


4.24.0


In [1]:
import os
#os.environ["WANDB_DISABLED"]="true" 

In [2]:
#imports section
import transformers
from datasets import load_dataset, load_metric
import datasets
import random
import pandas as pd
from IPython.display import display, HTML
from transformers import DataCollatorForSeq2Seq,AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
import numpy as np
import codecs
from datasets.dataset_dict import DatasetDict
from datasets import Dataset
from transformers import T5ForConditionalGeneration, T5Tokenizer
import csv
from collections import Counter
from tqdm.auto import tqdm, trange
import torch

In [3]:
#raw_datasets = load_dataset("wmt16", "ro-en") this ds 404s so likely isn't there
#we probably will want to use our own ds anyways
metric = load_metric("sacrebleu")

C:\Users\ewu15\AppData\Local\Temp\ipykernel_13088\2062494887.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")


In [4]:
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ru-en")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ru-en") 
#we kind of hope that the vocab is diverse enough. also that our untranslated texts are ok enough
model_checkpoint = "Helsinki-NLP/opus-mt-ru-en"

In [58]:
src_text = ['А вера — не вера — я не знаю, что это такое, — но чувство это так же незаметно вошло страданиями и твердо засело в душе.']
translated = model.generate(**tokenizer(src_text, return_tensors="pt", padding=True))
[tokenizer.decode(t, skip_special_tokens=True) for t in translated]

C:\Users\ewu15\anaconda3\lib\site-packages\transformers\generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 512 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


["And faith — not faith — I don't know what it is, but the feeling of it has also gone unnoticed and firmly inside the soul."]

In [5]:
def parsefile(directory, filename, outdir):
    file = open(f"{directory}/{filename}", encoding="utf8")
    txt = file.read()
    file.close() 
    split = txt.split(".")
    output = codecs.open(f"{outdir}/{filename}", 'a', "utf-8")
    #we can read and split very quickly but the tokenizer doesn't appreciate massive inputs
    for i in range(int(len(split) / 20)):
        cut = split[i * 20 : i * 20 + 20]
        tokens = tokenizer(cut, return_tensors="pt", padding=True, max_length=max_input_length, truncation=True)
        translated = model.generate(**tokens)
        #translation = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
        for t in translated:
            translation = tokenizer.decode(t, skip_special_tokens=True)
            output.write(translation + "\n")
    if (len(split) % 20 != 0):
        last = int(len(split) / 20)
        cut = split[last * 20 :]
        tokens = tokenizer(cut, return_tensors="pt", padding=True, max_length=max_input_length, truncation=True)
        translated = model.generate(**tokens)
        for t in translated:
            translation = tokenizer.decode(t, skip_special_tokens=True)
            output.write(translation + "\n")
    output.close()

In [16]:
#https://stackoverflow.com/questions/10377998/how-can-i-iterate-over-files-in-a-given-directory
max_input_length = 128
directory = "russian short stories 2"#"russian short stories"
#directory = "test translation"
files = ["Фокус великого кино averchenko.txt"]
#for file in os.listdir(directory):
for file in files:
    filename = os.fsdecode(file)
    if filename.endswith(".txt"):
        parsefile(directory, filename, "init translations")

In [68]:
prefix = ""
max_input_length = 128
max_target_length = 128
source_lang = "ru"
target_lang = "en"
def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    #pure dictionary version
    #inputs = [prefix + ex["translation"][source_lang] for ex in examples]
    #targets = [ex["translation"][target_lang] for ex in examples]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs
#as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. 
#You can tokenize your labels by using the argument `text_target` of the regular `__call__` method 
#(either in the same call as your input texts if you use the same keyword arguments, or in a separate call.

In [84]:
raw_datasets = {}
train_entries = []
entry = {}
translation = {}
pairs = [["crime and punishment ru.txt", "crime and punishment constance garnett.txt"], 
         ["the brothers karamazov ru.txt", "the brothers karamazov constance garnett.txt"]]
pairs = [["rutest.txt", "engtest.txt"]]##
pairs = [["ruvalid.txt", "engvalid.txt"]]##
for combination in pairs:
    first_loc = combination[0]
    second_loc = combination[1]
    f = codecs.open(first_loc,"r",encoding='utf-8')
    #read the file to unicode string
    rutrain = f.read()
    f.close()
    f = codecs.open(second_loc,"r",encoding='utf-8')
    entrain = f.read()
    f.close()
    translation["ru"] = rutrain
    translation["en"] = entrain
    #entry['translation'] = translation
    entry = translation
    train_entries.append(entry)
#entry['translation'] = train_entries
raw_datasets["train"] = train_entries
#the good news is that this does work
#the bad news is that we need to match next from russian and english
#that is beyond my skill set

In [85]:
first_loc = "rutest.txt"#"ruvalid.txt"
second_loc = "engtest.txt"#"engvalid.txt"
f = codecs.open(first_loc,"r",encoding='utf-8')
rutext = f.read()
f.close()
f = codecs.open(second_loc,"r",encoding='utf-8')
entext = f.read()
f.close()
#anna karenina has several parts each with chapters en prefaced by chapter 1 while russian is in roman numerals
#we try to send both as entire batches for now?
#raw_datasets = {}
valid_entries = []
entry = {}
translation = {}
translation["ru"] = rutext
translation["en"] = entext
#entry['translation'] = translation
entry = translation
valid_entries.append(entry)
#entry['translation'] = train_entries
raw_datasets["validation"] = valid_entries
#https://stackoverflow.com/questions/67852880/how-can-i-handle-this-datasets-to-create-a-datasetdict

dataset should be: not just a dict
DatasetDict({
    train: Dataset({
        features: ['translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 610320
    })
    validation: Dataset({
        features: ['translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1999
    })
    test: Dataset({
        features: ['translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1999
    })
})


In [86]:
d = {'train':Dataset.from_dict({'translation':train_entries}),
     'validation':Dataset.from_dict({'translation':valid_entries})
     }
#,'test':Dataset.from_dict({'label':y_test,'text':x_test})   ,'text':"fake"}
testdict = DatasetDict(d)
#print(testdict)
#print(testdict["train"][0]['translation']['ru'])#['translation'])#[0]['ru'][:55])#[0]['ru'][:55])
#print(testdict['translation'])
tokenized_datasets = testdict.map(preprocess_function, batched=True)
#tokenized_datasets = preprocess_function(raw_datasets)

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [87]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True    
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [88]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [74]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [82]:
#{'input_ids': [[393, 4462, 14, 1137, 53, 216, 28636, 0], [24385, 14, 28636, 14, 4646, 4622, 53, 216, 28636, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[42140, 494, 1750, 53, 8, 59, 903, 3543, 9, 15202, 0], [36199, 6612, 9, 15202, 122, 568, 35788, 21549, 53, 8, 59, 903, 3543, 9, 15202, 0]]}
print(tokenized_datasets['train']['input_ids'][0])#['train']['translation'][0])#['input_ids'][0])


[840, 12766, 30, 1940, 17876, 1897, 17, 4119, 2, 10660, 5539, 5313, 3284, 106, 8757, 5539, 5313, 12484, 15, 11, 20229, 504, 3, 840, 16, 4001, 41, 2559, 6, 7957, 4101, 8280, 2192, 3, 24121, 41, 8117, 41, 2, 28, 4577, 208, 6, 226, 16, 21, 48851, 30, 173, 6, 89, 7957, 4194, 3832, 7581, 4377, 1274, 173, 11, 4058, 571, 141, 5686, 1757, 2, 7, 28860, 4577, 60, 2, 28, 26, 223, 4732, 16, 1062, 6, 4481, 7957, 3, 5609, 79, 35322, 465, 10499, 1027, 7, 17591, 70, 15018, 2632, 7, 26610, 16756, 121, 2, 7, 2700, 3164, 1940, 2, 7, 3556, 2029, 4029, 8869, 3, 840, 2501, 1940, 7, 3556, 2029, 4029, 3091, 15018, 53, 2, 28, 482, 17153, 6, 89, 51662, 4590, 7, 28, 0]


In [89]:
trainer = Seq2SeqTrainer(
    model,
    args,
    #train_dataset=list(tokenized_datasets["train"]),
    #eval_dataset=list(tokenized_datasets["validation"]),
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [90]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `MarianMTModel.forward` and have been ignored: translation. If translation are not expected by `MarianMTModel.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 1
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1
  Number of trainable parameters = 76147712


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,No log,1.870704,28.804000,120.000000


The following columns in the evaluation set don't have a corresponding argument in `MarianMTModel.forward` and have been ignored: translation. If translation are not expected by `MarianMTModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1, training_loss=2.946575164794922, metrics={'train_runtime': 14.3573, 'train_samples_per_second': 0.07, 'train_steps_per_second': 0.07, 'total_flos': 33898364928.0, 'train_loss': 2.946575164794922, 'epoch': 1.0})

In [48]:
directory = "model-checkpoint"
trainer.save_model(directory)

Saving model checkpoint to model-checkpoint
Configuration saved in model-checkpoint\config.json
Model weights saved in model-checkpoint\pytorch_model.bin
tokenizer config file saved in model-checkpoint\tokenizer_config.json
Special tokens file saved in model-checkpoint\special_tokens_map.json


In [ ]:
for dirname, _, filenames in os.walk('opus-mt-en-ro-finetuned-en-to-ro'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [91]:
#from transformers import MarianMTModel, MarianTokenizer
#src_text = ['My name is Sarah and I live in London']
src_text = ['А вера — не вера — я не знаю, что это такое, — но чувство это так же незаметно вошло страданиями и твердо засело в душе.']
#model_name = 'opus-mt-en-ro-finetuned-en-to-ro/checkpoint-38000'
#tokenizer = MarianTokenizer.from_pretrained(model_name)
print(tokenizer.supported_language_codes)

[]


In [92]:
#model = MarianMTModel.from_pretrained(model_name)
translated = model.generate(**tokenizer(src_text, return_tensors="pt", padding=True))
[tokenizer.decode(t, skip_special_tokens=True) for t in translated]

["And faith — not faith — I don't know what it is, but the feeling of it has also gone undetected by suffering and is firmly rooted in the soul."]

In [ ]:
#https://stackoverflow.com/questions/10377998/how-can-i-iterate-over-files-in-a-given-directory
max_input_length = 128
directory = "russian short stories"
#directory = "test translation"
files = ["Сказка о лысом пророке Елисее chiornyi.txt"]
#for file in os.listdir(directory):
for file in files:
    filename = os.fsdecode(file)
    if filename.endswith(".txt"):
        parsefile(directory, filename, "tuned translations")

In [97]:
#model = T5ForConditionalGeneration.from_pretrained('t5-base')
#tokenizer = T5Tokenizer.from_pretrained('t5-base')
tokenizer = T5Tokenizer.from_pretrained("google/mt5-base")
model = T5ForConditionalGeneration.from_pretrained('google/mt5-base')

loading file spiece.model from cache at C:\Users\ewu15/.cache\huggingface\hub\models--google--mt5-base\snapshots\2eb15465c5dd7f72a8f7984306ad05ebc3dd1e1f\spiece.model
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at C:\Users\ewu15/.cache\huggingface\hub\models--google--mt5-base\snapshots\2eb15465c5dd7f72a8f7984306ad05ebc3dd1e1f\special_tokens_map.json
loading file tokenizer_config.json from cache at C:\Users\ewu15/.cache\huggingface\hub\models--google--mt5-base\snapshots\2eb15465c5dd7f72a8f7984306ad05ebc3dd1e1f\tokenizer_config.json
loading configuration file config.json from cache at C:\Users\ewu15/.cache\huggingface\hub\models--google--mt5-base\snapshots\2eb15465c5dd7f72a8f7984306ad05ebc3dd1e1f\config.json
Model config MT5Config {
  "_name_or_path": "google/mt5-base",
  "architectures": [
    "MT5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new"

In [96]:
text = "translate English to French: The quick brown fox jumps over the lazy dog"

# Tokenize the input
inputs = tokenizer.encode(text, return_tensors='pt')

# Generate translation
outputs = model.generate(inputs, max_length=40, num_beams=4, early_stopping=True)

# Decode the output
translated_text = tokenizer.decode(outputs[0])
print(translated_text)

<pad> <extra_id_0> The quick brown...</s>


In [94]:
text = "translate Russian to English: А вера — не вера — я не знаю"
inputs = tokenizer.encode(text, return_tensors='pt')
outputs = model.generate(inputs, max_length=40, num_beams=4, early_stopping=True)
translated_text = tokenizer.decode(outputs[0])

print(translated_text)

<pad> <extra_id_0>. )</s>


In [98]:
def msize(m):
    return sum(p.numel() for p in m.parameters())
print(msize(model.shared) / msize(model))   # 0.3298
print(msize(model.lm_head) / msize(model))  # 0.3298

0.32981729710484153
0.32981729710484153


In [99]:
df_ru = pd.read_csv('rus-ru_web-public_2019_10K-sentences.txt', sep='\t', header=None, quoting=csv.QUOTE_NONE)
df_ru.columns = ['idx', 'text']
cnt_ru = Counter()
for text in tqdm(df_ru.text):
    cnt_ru.update(tokenizer.encode(text))
print(len(cnt_ru), len(cnt_ru)/tokenizer.vocab_size)  
# 58438 0.2336

  0%|          | 0/10000 [00:00<?, ?it/s]

16528 0.06608556577369053


In [100]:
for top in 1000, 2000, 3000:
    print(top, sum(v for k, v in cnt_ru.most_common(top)) / sum(cnt_ru.values()))
# 10000 0.9645
# 20000 0.9940
# 30000 0.9982

1000 0.7193085003902258
2000 0.7999464460067988
3000 0.8492389781636859


In [101]:
cnt_en = Counter()

new_tokens = set(range(1000))
for i, (k, v) in enumerate(cnt_en.most_common(10_000)):
    if k not in new_tokens:
        new_tokens.add(k)
for i, (k, v) in enumerate(cnt_ru.most_common(25_000)):
    if len(new_tokens) == 29_900:
        print(i, 'Russan tokens are included')
        break
    if k not in new_tokens:
        new_tokens.add(k)
for t in range(tokenizer.vocab_size - 100, tokenizer.vocab_size):
    new_tokens.add(t)
print(len(new_tokens))
kept_ids = sorted(new_tokens)

17332


In [102]:
new_size = len(kept_ids)
new_emb = torch.nn.Embedding(new_size, model.shared.embedding_dim)
new_head = torch.nn.Linear(in_features=model.lm_head.in_features, out_features=new_size, bias=False)
for new_id, old_id in enumerate(kept_ids):
    new_emb.weight.data[new_id] = model.shared.weight.data[old_id]
    new_head.weight.data[new_id] = model.lm_head.weight.data[old_id]
model.shared.weight = new_emb.weight
model.lm_head.weight = new_head.weight
model.config.__dict__['vocab_size'] = new_size
model.config.__dict__['_name_or_path'] = 'cointegrated/rut5-base'

In [ ]:
smp = tokenizer.sp_model.serialized_model_proto()
m = spmp.ModelProto()
m.ParseFromString(smp)
print('the loaded model has pieces:', len(m.pieces))
new_pieces = [m.pieces[idx] for idx in kept_ids]
print('the new pieces:', len(new_pieces))
# replace the content of the first 30K pieces
for i, p in enumerate(new_pieces):
    m.pieces[i].piece = p.piece
    m.pieces[i].score = p.score
    m.pieces[i].type = p.type
# drop the remaining pieces
n = len(new_pieces)
for i in trange(len(m.pieces) - n):
    m.pieces.pop(len(m.pieces) - 1)
print(len(m.pieces))
with open('new_sp.model', 'wb') as f:
    f.write(m.SerializeToString())
new_tokenizer = T5Tokenizer('new_sp.model', extra_ids=0)

In [ ]:
new_tokenizer.save_pretrained('rut5-base')
model.save_pretrained('rut5-base')

In [ ]:
#google translations done manually. bing translation done similarly

In [ ]:
#https://stackoverflow.com/questions/10377998/how-can-i-iterate-over-files-in-a-given-directory
max_input_length = 128
directory = "russian short stories"
directory = "test translation"
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".txt"):
        file = open(f"{directory}/{filename}", encoding="utf8")
        txt = file.read()
        file.close() 
        split = txt.split(".")
        #might need to break text into sentences due to model's limits
        #tokens = tokenizer(txt, return_tensors="pt", padding=True, max_length=max_input_length, truncation=True)
        tokens = tokenizer(split, return_tensors="pt", padding=True, max_length=max_input_length, truncation=True)
        #print(tokens['input_ids'][:20])
        #print(tokens)
        translated = model.generate(**tokens)
        #translated = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
                #print(translated)
                #whole += translated[0]
        print(translated.shape)
        output = codecs.open(f"init translations/{filename}", 'a', "utf-8")
        
        translation = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
        #print(len(translation))
        for string in translation:
            output.write(string)
            print(string)
        #for i in range(int(translated.shape[1] / 20)):
            #translation = [tokenizer.decode(t, skip_special_tokens=True) for t in translated[i * 20 : i * 20 + 20]][0]
        #print(translation)
        #print(filename)
            #output.write(translation)
        #if (translated.shape[1] % 20 != 0):
            #last = int(translated.shape[1] / 20)
            #translation = [tokenizer.decode(t, skip_special_tokens=True) for t in translated[last * 20 :]][0]
        output.close()

In [41]:
first_loc = "rutest.txt"
second_loc = "engtest.txt"
#open it with utf-8 encoding 
f = codecs.open(first_loc,"r",encoding='utf-8')
#read the file to unicode string
rutrain = f.read()
f.close()
f = codecs.open(second_loc,"r",encoding='utf-8')
#read the file to unicode string
entrain = f.read()
f.close()
#print(rutext)
raw_datasets = {}
train_entries = []
entry = {}
translation = {}
rutrains = []
entrains = []
#for ensource in ensources:
f = codecs.open(ensource,"r",encoding='utf-8')
entrain = f.read()
f.close()
    entrains.append(entrain)
for rusource in rusources:
    f = codecs.open(rusource,"r",encoding='utf-8')
    rutrain = f.read()
    f.close()
    rutrains.append(rutrain)
translation["ru"] = rutrains
translation["en"] = entrains
#entry["translation"] = translation
entry = translation
train_entries.append(entry)
raw_datasets["train"] = train_entries

IndentationError: unexpected indent (1361443166.py, line 23)

In [ ]:
#https://stackoverflow.com/questions/10376923/reading-non-ascii-characters-from-a-text-file
#will need to do for all texts which include anna karenina, crime and punishment, the brothers
first_loc = "anna karenina ru.txt"
second_loc = "anna karenina translaated by constance garnett.txt"
f = codecs.open(first_loc,"r",encoding='utf-8')
rutext = f.read()
f.close()
f = codecs.open(second_loc,"r",encoding='utf-8')
entext = f.read()
f.close()
#anna karenina has several parts each with chapters en prefaced by chapter 1 while russian is in roman numerals
#we try to send both as entire batches for now?
raw_datasets = {}
train_entries = []
entry = {}
translation = {}
translation["ru"] = rutext
translation["en"] = entext
#entry["translation"] = translation
entry = translation
train_entries.append(entry)
raw_datasets["train"] = train_entries


In [ ]:
raw_datasets = {}
train_entries = []
entry = {}
translation = {}
pairs = [["crime and punishment ru.txt", "crime and punishment constance garnett.txt"], 
         ["the brothers karamazov ru.txt", "the brothers karamazov constance garnett.txt"]]
for combination in pairs:
    first_loc = combination[0]
    second_loc = combination[1]
    f = codecs.open(first_loc,"r",encoding='utf-8')
    #read the file to unicode string
    rutrain = f.read()
    f.close()
    f = codecs.open(second_loc,"r",encoding='utf-8')
    entrain = f.read()
    f.close()
    translation["ru"] = rutrain
    translation["en"] = entrain
    train_entries.append(translation)
entry['translation'] = train_entries
raw_datasets["train"] = entry

In [ ]:
prefix = ""
max_input_length = 128
max_target_length = 128
source_lang = "ru"
target_lang = "en"
def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
#print(raw_datasets['train'][0])
#preprocess_function(raw_datasets['train'][0])
#tokenized_datasets = raw_datasets['train'].map(preprocess_function, batched=True)
#wrong
tokenized_datasets = {}
tokenized_datasets['train'] = preprocess_function(raw_datasets['train'])#, batched=True)
tokenized_datasets['validation'] = preprocess_function(raw_datasets['validation'])